# Análise Completa de Tempo de Execução e Profiling (Pós-Vetorização)

Este notebook analisa os resultados de tempo de execução e profiling do script `main_script_sp_timing_estimate.py` após a implementação de duas fases principais de otimização:
1. Redução do uso de `deepcopy` no algoritmo Simulated Annealing (SA).
2. Vetorização das funções `is_valid()` e `fitness()` utilizando NumPy, e adaptação dos algoritmos para usar estas versões.

O objetivo é avaliar o impacto combinado destas otimizações no desempenho geral do script e nos tempos de execução de cada algoritmo.

## 1. Resultados Gerais de Tempo de Execução

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Tempos de execução (aproximados, baseados nos logs)
tempo_original_total = 85.66  # s, antes de qualquer otimização
tempo_pos_deepcopy_opt_total = 41.42  # s, após otimização de deepcopy no SA
tempo_pos_vectorizacao_total = 40.96  # s, após vetorização de is_valid/fitness

tempos_sa_original = 73.17 # s
tempos_sa_pos_deepcopy_opt = 28.91 # s
tempos_sa_pos_vectorizacao = 28.98 # s (aproximado do log de profiling para a função simulated_annealing)

print(f"Tempo total original do script: {tempo_original_total:.2f}s")
print(f"Tempo total após otimização de deepcopy no SA: {tempo_pos_deepcopy_opt_total:.2f}s")
print(f"Tempo total após vetorização de is_valid/fitness: {tempo_pos_vectorizacao_total:.2f}s")

print(f'Melhoria total em relação ao original: {tempo_original_total - tempo_pos_vectorizacao_total:.2f}s ({( (tempo_original_total - tempo_pos_vectorizacao_total) / tempo_original_total) * 100:.2f}%)')
print(f'Melhoria da vetorização em relação à otimização de deepcopy: {tempo_pos_deepcopy_opt_total - tempo_pos_vectorizacao_total:.2f}s')

Observamos que o tempo total do script foi reduzido de **85.66s** para **40.96s**, uma melhoria total de aproximadamente **52.18%**. A maior parte desta melhoria veio da otimização do `deepcopy` no Simulated Annealing. A vetorização das funções `is_valid` e `fitness` resultou numa pequena melhoria adicional no tempo total do script (de 41.42s para 40.96s).

## 2. Tempos de Execução por Algoritmo (Pós-Vetorização)

In [ ]:
# Tempos aproximados das seções dos algoritmos (do log da última execução)
tempo_hc_vectorized = 0.68  # s (tempo da função hill_climbing no profile)
tempo_sa_vectorized = 28.98  # s (tempo da função simulated_annealing no profile)
tempo_ga_vectorized = 5.51  # s (tempo da função genetic_algorithm no profile, para todas as configs)

print(f"Tempo do Hill Climbing (HC) pós-vetorização: {tempo_hc_vectorized:.2f}s")
print(f"Tempo do Simulated Annealing (SA) pós-vetorização: {tempo_sa_vectorized:.2f}s")
print(f"Tempo dos Algoritmos Genéticos (GAs) pós-vetorização (total para 9 configs, 1 geração): {tempo_ga_vectorized:.2f}s")

# Comparação do SA
print(f'Tempo SA original: {tempos_sa_original:.2f}s')
print(f'Tempo SA após otimização deepcopy: {tempos_sa_pos_deepcopy_opt:.2f}s')
print(f'Tempo SA após vetorização: {tempos_sa_vectorized:.2f}s')

O tempo do Simulated Annealing (SA) permaneceu muito similar após a vetorização (28.91s vs 28.98s), sugerindo que a vetorização de `is_valid` e `fitness` não trouxe um benefício significativo adicional para o SA, ou que possíveis overheads do NumPy para chamadas individuais anularam os ganhos. O Hill Climbing e os Algoritmos Genéticos são inerentemente mais rápidos com os parâmetros atuais (1 geração para AGs).

## 3. Análise Detalhada do Profiling (Pós-Vetorização)

Analisamos o ficheiro `profile_analysis.txt` gerado pela última execução.

In [ ]:
# Dados chave do profiling (baseado no profile_analysis.txt mais recente)
profiling_data = {
    'is_valid': {'total_time': 22.556, 'n_calls': 201024},
    'fitness': {'total_time': 13.361, 'n_calls': 73856},
    'get_random_neighbor_SA': {'total_time': 20.213, 'n_calls': 45850}, # solution.py:149(get_random_neighbor)
    'deepcopy': {'total_time': 0.157, 'n_calls': '39767/1251'}, # copy.py:128(deepcopy)
    'simulated_annealing_func': {'total_time': 28.984, 'n_calls': 1}, # evolution.py:165(simulated_annealing)
    'genetic_algorithm_func': {'total_time': 5.514, 'n_calls': 9} # evolution.py:45(genetic_algorithm)
}

print("Principais Funções por Tempo Cumulativo (Pós-Vetorização):")
for func, data in profiling_data.items():
    print(f"  {func}: Tempo Total = {data['total_time']:.3f}s, Chamadas = {data['n_calls']}")

# Comparação com profiling anterior (pós-deepcopy, antes de vetorizar)
# Dados do profiling anterior (aproximados, para comparação)
profiling_data_prev = {
    'is_valid': {'total_time': 22.018, 'n_calls': 201024}, # Antes da vetorização, mas com otimização de deepcopy no SA
    'fitness': {'total_time': 13.110, 'n_calls': 73856}, # Antes da vetorização
    'get_random_neighbor_SA': {'total_time': 19.876, 'n_calls': 45850}, # Antes da vetorização
    'deepcopy_geral_SA_opt': {'total_time': 0.168, 'n_calls': '39k/1k'} # Estimativa após otimização de deepcopy no SA
}

print("
Comparação do Tempo de Funções Chave (Vetorizado vs. Pós-Deepcopy Opt.):")
print(f"  is_valid: {profiling_data['is_valid']['total_time']:.3f}s (vetorizado) vs. {profiling_data_prev['is_valid']['total_time']:.3f}s (anterior)")
print(f"  fitness: {profiling_data['fitness']['total_time']:.3f}s (vetorizado) vs. {profiling_data_prev['fitness']['total_time']:.3f}s (anterior)")
print(f"  get_random_neighbor (SA): {profiling_data['get_random_neighbor_SA']['total_time']:.3f}s (vetorizado) vs. {profiling_data_prev['get_random_neighbor_SA']['total_time']:.3f}s (anterior)")

**Observações do Profiling Pós-Vetorização:**

*   **`is_valid()` e `fitness()`:** Estas funções continuam a consumir uma porção significativa do tempo (22.56s e 13.36s, respetivamente). Comparando com os tempos antes da vetorização (mas após a otimização do `deepcopy` no SA), os tempos são muito similares (22.02s para `is_valid` e 13.11s para `fitness`). Isto sugere que a vetorização, na forma como foi implementada e como estas funções são chamadas (tipicamente uma solução de cada vez dentro dos loops dos algoritmos), não resultou numa redução drástica do tempo de execução para estas funções específicas. É possível que o overhead das operações NumPy para arrays pequenos ou chamadas individuais esteja a anular os benefícios da vetorização, ou que a lógica interna ainda tenha partes não otimizadas.
*   **`get_random_neighbor()` (SA):** Continua a ser uma função dispendiosa (20.21s). Isto é esperado, pois a sua otimização principal (remover o `deepcopy` interno) ainda não foi abordada.
*   **`deepcopy`:** O tempo gasto em `deepcopy` (0.157s) permanece baixo, confirmando que as otimizações anteriores no SA foram eficazes e que a vetorização não reintroduziu uso excessivo de `deepcopy` de forma global.
*   **Simulated Annealing (`simulated_annealing_func`):** O tempo total da função (28.98s) é consistente com a soma dos seus componentes principais (`is_valid`, `fitness`, `get_random_neighbor`).

## 4. Conclusões e Próximos Passos Sugeridos

1.  **Impacto da Vetorização:** A vetorização das funções `is_valid` e `fitness` não produziu a melhoria de desempenho substancial que se poderia esperar, especialmente para o Simulated Annealing. Os tempos de execução destas funções permaneceram muito semelhantes aos da versão não vetorizada (mas com o SA já otimizado para `deepcopy`). Isto pode dever-se ao facto de estas funções serem frequentemente chamadas com soluções individuais, onde o overhead do NumPy pode ser comparável aos ganhos, ou a estrutura interna das funções vetorizadas ainda poder ser melhorada.

2.  **Principal Gargalo Restante no SA:** A função `get_random_neighbor` dentro do `LeagueSASolution` (que ainda utiliza `deepcopy` internamente para criar o `candidate_assignment`) é agora o principal alvo para otimizações adicionais no SA, juntamente com o elevado número de chamadas a `is_valid` e `fitness`. 

**Próximos Passos Sugeridos:**

*   **Otimizar `get_random_neighbor` no SA:** Implementar a técnica de "modificar e reverter" para evitar o `deepcopy` interno nesta função. Esta é provavelmente a otimização com maior potencial de ganho restante para o SA.
*   **Rever Implementação da Vetorização:** Analisar se a implementação atual da vetorização em `is_valid` e `fitness` é a mais eficiente possível para o padrão de uso no código. Poderia haver formas de reestruturar os algoritmos para tirar melhor partido de operações em batch, se aplicável.
*   **Ajustar Parâmetros do SA:** Considerar reduzir o número de `iterations_per_temp` ou ajustar `alpha` no SA para diminuir o número total de avaliações, o que reduziria o tempo gasto em `is_valid` e `fitness`, embora possa impactar a qualidade da solução.
*   **Foco noutros Algoritmos:** Se o desempenho do SA, mesmo após mais otimizações, não for considerado suficiente, pode ser necessário focar em otimizar os AGs (por exemplo, executando por mais gerações e analisando o seu tempo) ou aceitar o HC como a opção mais rápida para uma solução rápida, embora potencialmente subótima.